# OpenFEMA API Tutorial: Part 4 - Metadata
  
## Quick Summary
- This tutorial explains and demonstrates the use of the special OpenFEMA metadata API endpoints. 
- These endpoints can be useful in accessing data about OpenFEMA datasets such as the last time a dataset was refreshed.
- OpenFEMA metadata conform to specifications to promote the standardization of metadata.
- Several examples are presented to:
  - List all active OpenFEMA (Non-Deprecated) Datasets
  - List Deprecated Datasets
  - Get Basic Dataset Overview
  - Identify a Dataset's Primary Keys
  - Check for Updates to a Dataset
  - Produce Valid Data Definition Language (DDL) to Create a Database Table
  - List Codes for a Dataset, Dataset Field, and Dataset Value
- The next couple tutorials will cover basic data analysis, graphing, and mapping.

## Overview
In previous tutorials, we touched on metadata that is returned with a dataset query. While that metadata describes the call (e.g., the criteria used, fields returned, date run) and sometimes returns messages about the dataset (e.g., deprecation information), the metadata presented in this tutorial describes the dataset itself (e.g., description, update frequency, field definitions). The dataset webpages on fema.gov/open display some of this information, but there are other data that are only available through the metadata endpoints ([OpenFemaDataSets](https://www.fema.gov/openfema-data-page/openfema-data-sets-v1) and [OpenFemaDataSetFields](https://www.fema.gov/openfema-data-page/openfema-data-set-fields-v1)). Some of the metadata items were included to conform to [DCAT-US Schema v1.1 (Project Open Data Metadata Schema)](https://resources.data.gov/schemas/dcat-us/v1.1/). This is the standard used by the [Data.gov](https://www.data.gov/) project to harvest and aggregate metadata from various federal agencies. Data.gov acts as a public, central repository for federal data *(metadata only, not the actual datasets)* and is essentially a discoverability tool. Further, the OpenFEMA metadata schema also aligns nicely with the [Dublin Core Metadata Initiative](https://www.dublincore.org/specifications/dublin-core/), another specification to promote the standardization of metadata.

The metadata endpoints can be used to find information about the datasets, such as getting a list of all OpenFEMA datasets that are not deprecated. In addition, the metadata endpoints are especially useful if you are automating access to the datasets or incorporating the API into your own product. Being able to check for updates or dynamically present a field description are good examples of metadata usage.

The purpose of this notebook is to demonstrate the use of these metadata endpoints and to suggest possible use cases. The examples are presented using Python 3, but it should be easy to translate them to almost any programming language.


## OpenAPI Specification
An API specification defines how an API functions and the results to expect when using the API. Usually, API specifications are associated with documentation. They do more however, providing an understanding of API behavior especially with regard to links with other API’s. Specifications define machine-readable interface files for describing, producing, consuming, and visualizing RESTful web services. Once defined, tools exist to generate documentation, code stubs, and test cases for a given interface.

In addition to the “official” OpenFEMA API documentation available on fema.gov and the two special metadata endpoints described below, an endpoint has been created to provide an [OpenAPI Specification v3.0](https://spec.openapis.org/oas/v3.0.0) file. This format will allow for the experimentation and testing of the OpenFEMA API endpoints directly from a visual interface.

The base path for the OpenAPI metadata endpoint is: https://www.fema.gov/api/open/metadata/v3.0/OpenApi

A file type suffix must be added to the base path. Supported file types are:
 - json – returns data in a JSON format
 - yaml – returns data in a YAML format

Example of use: https://www.fema.gov/api/open/metadata/v3.0/OpenApi.json


## Metadata Endpoints

### OpenFemaDataSets (a.k.a. DataSets)

<div class="alert alert-block alert-info">
    <b>Tip:</b> The actual endpoint names are *DataSets* and *DataSetFields*. To clarify that these endpoints provided data about OpenFEMA, the fema.gov dataset page names were prefixed with OpenFema. OpenFemaDataSets and OpenFemaDataSetFields are aliases for the original endpoints. Either may be used. The examples in this tutorial will used the shortened version.
</div>

The metadata API endpoint that describes each OpenFEMA dataset is: [OpenFemaDataSets](https://www.fema.gov/openfema-data-page/openfema-data-sets-v1) (a.k.a. DataSets). This dataset page describes the attributes of the DataSets dataset, including the fields it contains, so we won't list them all here. Many of the attributes are included to conform to with our guiding metadata standards. Some useful attributes include: the dataset name, its version, a description, update frequency, the last update, and deprecation information.

Let's call the DataSets endpoint for the DataSets metadata dataset itself. How meta!

In [1]:
# declare a URL handling module
import urllib.request
import json

# define URL for the Data Sets endpoint 
baseUrl = "https://www.fema.gov/api/open/v1/DataSets"

# limit to just the Datasets dataset
filter = "&$filter=name%20eq%20%27DataSets%27"    

# we don't really need to see the query metadata
other = "&$metadata=off"   

# define the request and read the data 
request = urllib.request.urlopen(baseUrl + "?" + filter + other)
result = request.read()

# transform result to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# display the results
print(json.dumps(jsonData, indent=2))

{
  "DataSets": [
    {
      "identifier": "openfema-7",
      "name": "DataSets",
      "title": "OpenFEMA Data Sets",
      "description": "Metadata for the OpenFEMA API data sets. It contains attributes regarding the published datasets including but not limited to update frequency, description, version, and deprecation status.\n\nIf you have media inquiries about this dataset please email the FEMA News Desk FEMA-News-Desk@dhs.gov or call (202) 646-3272.  For inquiries about FEMA's data and Open government program please contact the OpenFEMA team via email OpenFEMA@fema.dhs.gov.",
      "distribution": [
        {
          "format": "csv",
          "accessURL": "https://www.fema.gov/api/open/v1/OpenFemaDataSets.csv",
          "datasetSize": "tiny (< 10MB)"
        },
        {
          "format": "json",
          "accessURL": "https://www.fema.gov/api/open/v1/OpenFemaDataSets.json",
          "datasetSize": "tiny (< 10MB)"
        },
        {
          "format": "jsona",
      

Let's run this for an easier to understand dataset - Disaster Declarations Summaries - and retrieve some of the more important attributes rather than the full set.

In [2]:
# limit to just the DisasterDeclarationsSummaries dataset
filter = "&$filter=name%20eq%20%27DisasterDeclarationsSummaries%27"    

# pull many of the more important attributes
select = "&$select=name,version,description,webService,dataDictionary,accrualPeriodicity,lastRefresh,depDate,depApiMessage,recordCount"

# define the request and read the data 
request = urllib.request.urlopen(baseUrl + "?" + filter + select + other)
result = request.read()

# transform result to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# display the results
print(json.dumps(jsonData, indent=2))

{
  "DataSets": [
    {
      "name": "DisasterDeclarationsSummaries",
      "version": 2,
      "description": "FEMA Disaster Declarations Summary is a summarized dataset describing all federally declared disasters. This dataset lists all official FEMA Disaster Declarations, beginning with the first disaster declaration in 1953 and features all three disaster declaration types: major disaster, emergency, and fire management assistance. The dataset includes declared recovery programs and geographic areas (county not available before 1964; Fire Management records are considered partial due to historical nature of the dataset).\n\nPlease note the unique structure of the disaster sequencing (due to a numbering system that originated in the 1950's-1970's):\n0001-1999     Major Disaster Declaration\n2000-2999     Fire Management\n3000-3999     Emergency Declaration (Special Emergency)\n4000-4999     Major Disaster Declaration\n5000-5999     Fire Management\n\nFor more information on the dis

A key observation of the above query is that more than one record was returned. Two working endpoints of the Disaster Declarations Summaries dataset exist - version 1 and 2. Version 1 happens to be deprecated (and should no longer be used), but is still operational for a period of time to allow developers to transition to the new version. We could modify the filter above to limit results to the most recent version. Being unaware of multiple versions, modify the query to sort by version and pull the first record: ```"&$orderby=version%20desc&$top=1"```

<div class="alert alert-block alert-warning">
    <b>Note:</b> The above example is old. The V1 version has reached end-of-life and has been removed. Running the code above will not produce 2 results. But the point remains - there could be multiple versions, so specify criteria or sort to select the one you want.
</div>

### OpenFemaDataSetFields (a.k.a. DatasetFields)

The metadata API endpoint that describes the OpenFEMA dataset fields is: [OpenFemaDataSetFields](https://www.fema.gov/openfema-data-page/openfema-data-set-fields-v1) (a.k.a. DataSetFields). This dataset page describes the attributes of the DataSetFields dataset, including the fields it contains, so we won't list them all here. The attributes included are what you would expect from a data dictionary: field name, title, description, datatype, nullability, etc.

<div class="alert alert-block alert-info">
    <b>Tip:</b> To see all the elements this metadata endpoint returns, review the <a href="https://www.fema.gov/openfema-data-page/openfema-data-sets-v1">data dictionary page</a>.
</div>

As we did with the DataSets endpoint, let's call the DataSetFields endpoint *for the DataSetFields dataset itself* to review its field attributes.

In [3]:
# define URL for the Data Set Fields endpoint 
baseUrl = "https://www.fema.gov/api/open/v1/DataSetFields"

# limit to just the DataSetFields dataset
filter = "&$filter=openFemaDataSet%20eq%20%27DataSetFields%27"    

# we don't really need to see the query metadata
other = "&$metadata=off"   

# define the request and read the data 
request = urllib.request.urlopen(baseUrl + "?" + filter + other)
result = request.read()

# transform result to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# display the results
print(json.dumps(jsonData, indent=2))

{
  "DataSetFields": [
    {
      "datasetId": "openfema-8",
      "openFemaDataSet": "DataSetFields",
      "datasetVersion": 1,
      "name": "title",
      "title": "Title",
      "description": "Display title of the field.",
      "type": "string",
      "sortOrder": 5,
      "isSearchable": true,
      "isNestedObject": false,
      "isNullable": false,
      "primaryKey": false,
      "hash": "1d4bffc60ba95a4310e45a517ae347474aab6959",
      "id": "7853f63c-4995-4643-aaf9-0b1c246af936",
      "lastRefresh": "2022-07-20T22:51:38.902Z"
    },
    {
      "datasetId": "openfema-8",
      "openFemaDataSet": "DataSetFields",
      "datasetVersion": 1,
      "name": "lastRefresh",
      "title": "Last Refresh",
      "description": "Last date the dataset was updated in the datastore",
      "type": "date",
      "sortOrder": 14,
      "isSearchable": true,
      "isNestedObject": false,
      "isNullable": false,
      "primaryKey": false,
      "hash": "b50d2ca7471845f8642a52413ee1ae

Again, let's run this for an easier to understand dataset - Disaster Declarations Summaries. As we learned in our earlier example, this dataset has multiple versions, so add criteria to retrieve the latest version.

In [4]:
# define URL for the DataSetFields endpoint
baseUrl = "https://www.fema.gov/api/open/v1/DataSetFields"

# limit to just the DisasterDeclarationsSummaries dataset
filter = "&$filter=openFemaDataSet%20eq%20%27DisasterDeclarationsSummaries%27%20and%20datasetVersion%20eq%202"    

# no select - return all the attributes
select = ""

# define the request and read the data 
request = urllib.request.urlopen(baseUrl + "?" + filter + select + other)
result = request.read()

# transform result to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

#show the results
print(json.dumps(jsonData, indent=2))

{
  "DataSetFields": [
    {
      "datasetId": "openfema-47",
      "openFemaDataSet": "DisasterDeclarationsSummaries",
      "datasetVersion": 2,
      "name": "fyDeclared",
      "title": "FY Declared",
      "description": "Fiscal year in which the disaster was declared",
      "type": "smallint",
      "sortOrder": 6,
      "isSearchable": true,
      "isNestedObject": false,
      "isNullable": false,
      "primaryKey": false,
      "hash": "354f1784714c203acede995d163993b1ef57ca0f",
      "id": "42a4dc71-e141-4164-8295-f9f83be9e168",
      "lastRefresh": "2022-11-29T00:14:22.065Z"
    },
    {
      "datasetId": "openfema-47",
      "openFemaDataSet": "DisasterDeclarationsSummaries",
      "datasetVersion": 2,
      "name": "declarationType",
      "title": "Declaration Type",
      "description": "Two character code that defines if this is a major disaster, fire management, or emergency declaration. For more information on the disaster process, please visit https://www.fema.go

### DataSetCodes

Many of the OpenFEMA datasets return codes instead of detailed descriptions. Sometimes this is due to the way in which data is received from a source system; sometimes only codes are returned to reduce the size of the final dataset. While the code values are listed in the data dictionary, providing code descriptions in a format that can be queried is beneficial.

The metadata API endpoint that contains the OpenFEMA dataset codes is: [DataSetCodes](https://www.fema.gov/openfema-data-page/openfema-dataset-codes-v1). This dataset page describes the attributes of the DataSetCodes dataset, including the fields it contains, so we won't list them all here. The attributes included are what you would expect: dataset name, version, field name, code, and description.

Let's call the DataSetCodes endpoint and see what it returns:

In [5]:
# declare a URL handling module
import urllib.request
import json

# define URL for the Data Set Codes endpoint 
baseUrl = "https://www.fema.gov/api/open/v1/DataSetCodes"

# limit to only 3 records and hide the metadata
filter = "$top=3&$metadata=off"    

# define the request and read the data 
request = urllib.request.urlopen(baseUrl + "?" + filter)
result = request.read()

# transform result to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# display the results
print(json.dumps(jsonData, indent=2))

{
  "DataSetCodes": [
    {
      "datasetId": "openfema-35",
      "datasetName": "IpawsArchivedAlerts",
      "datasetVersion": 1,
      "fieldName": "category",
      "code": "Other",
      "description": "Other events, often Civil/Administrative/Special/Test",
      "id": "bb074282-8127-44d3-9d4b-3e6ab183212d",
      "lastRefresh": "2025-03-05T19:40:45.115Z",
      "hash": "2f7788d05d7099031fd753ab4369cb24bad4a107"
    },
    {
      "datasetId": "openfema-35",
      "datasetName": "IpawsArchivedAlerts",
      "datasetVersion": 1,
      "fieldName": "category",
      "code": "Safety",
      "description": "General emergency and public safety, sometimes Civil/Administrative/Special",
      "id": "db8cb9ff-e456-4be7-885a-80b2995669a5",
      "lastRefresh": "2025-03-05T19:40:45.115Z",
      "hash": "3e3d83a3aa3e4816d64b4075bf771aa49ab4b95f"
    },
    {
      "datasetId": "openfema-35",
      "datasetName": "IpawsArchivedAlerts",
      "datasetVersion": 1,
      "fieldName": "category

The real power of this metadata endpoint is to return a list of codes for a field or value within a specific dataset. Examples are shown in the sections below.

## Uses and Examples

<div class="alert alert-block alert-warning">
    <b>Note:</b> This is not meant to be a Python language tutorial. The point is to show several useful features of the metadata functions. There are other, more Pythonic ways that this can be done. If you are writing production quality code, it is recommended that you follow industry best practices - replace embedded constants, evaluate returned values, add error/exception handling, add logging, proper object cleanup, build for resilience by adding retries if failure, etc.
</div>

### Get Basic Dataset Overview
The DisasterDeclarationsSummaries example above illustrates this use. Similar to what is displayed on the individual dataset pages, a description, field titles, update frequency and other information can be retrieved. **A key takeaway is that the metadata endpoints contain more information than is displayed on the dataset pages.**

The fieldnames returned by the metadata endpoints (named such that they conform to metadata standards) will differ from the terminology used on the dataset pages (named to make it easier for the general public to understand). A crosswalk follows:

| Metadata Field Name | Dataset Webpage Name |
| :- | :- |
| DataSets.dataDictionary | url to page |
| DataSets.title | page title |
| DataSets.version | Version |
| DataSets.lastDataSetRefresh | Last Data Refresh |
| DataSets.name | Entity Name |
| DataSets.webService | API Endpoint |
| DataSets.accrualPeriodicity | Update Frequency |
| DataSets.landingPage | Program URL |
| DataSets.theme | Category |
| DataSets.keyword | Keywords |
| DataSets.depDate | Deprecation Date (only shows if deprecated) |
| DataSets.depWebMessage | Deprecation Message (only shows if deprecated) |
| DataSets.depNewURL | Deprecation New URL (only shows if deprecated) |
| DataSets.description | Description |
| DataSets.distribution.format | Full Data - Format |
| DataSets.distribution.accessURL | Full Data - Address |
| DataSets.recordCount | Full Data - Record Count |
| DataSets.distribution.datasetSize | Full Data - Approximate File Size |
| DataSets.references | References (only shows if references exist) |
| DataSetFields.name | Data Fields - Name |
| DataSetFields.title | Data Fields - Title |
| DataSetFields.type | Data Fields - Type |
| DataSetFields.description | Data Fields - Description |
| DataSetFields.isSearchable | Data Fields - Is Searchable |


### List all the OpenFEMA Active (Non-Deprecated) Datasets
A useful query to get a full list of datasets.

In [15]:
# declare a URL handling module
import urllib.request
import json

# define URL for the Data Sets endpoint 
baseUrl = "https://www.fema.gov/api/open/v1/DataSets"

# exclude deprecated datasets
filter = "&$filter=depDate%20eq%20null"    

# some of the more useful attributes
select = "&$select=name,version,theme,lastRefresh"

# we don't really need to see the query metadata, but let's at least sort by category and name
other = "&$metadata=off&$orderby=theme,name"   

# define the request and read the data 
request = urllib.request.urlopen(baseUrl + "?" + select + filter + other)
result = request.read()

# transform result to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# display as a list for readability
for rec in jsonData['DataSets']:
    print( rec['theme'] + ', ' + rec['name'] + ', ' + str(rec['version']) + ', ' + rec['lastRefresh'])

Disaster Information, DeclarationDenials, 1, 2022-12-24T06:29:47.658Z
Disaster Information, DisasterDeclarationsSummaries, 2, 2022-12-27T22:02:07.216Z
Disaster Information, EmergencyManagementPerformanceGrants, 2, 2022-10-19T19:22:03.262Z
Disaster Information, FemaWebDeclarationAreas, 1, 2022-12-27T22:04:35.325Z
Disaster Information, FemaWebDisasterDeclarations, 1, 2022-12-27T22:02:08.722Z
Disaster Information, FemaWebDisasterSummaries, 1, 2022-12-27T22:02:45.138Z
Disaster Information, MissionAssignments, 1, 2022-12-27T17:10:51.152Z
Emergency Management/Preparedness/Alerts, IpawsArchivedAlerts, 1, 2022-12-27T22:06:58.092Z
Emergency Management/Preparedness/Alerts, NonDisasterAssistanceFirefighterGrants, 1, 2022-12-16T01:44:15.959Z
Hazard Mitigation, HazardMitigationAssistanceMitigatedProperties, 3, 2022-12-27T17:10:25.056Z
Hazard Mitigation, HazardMitigationAssistanceProjects, 3, 2022-12-27T17:09:46.226Z
Hazard Mitigation, HazardMitigationAssistanceProjectsByNfipCrsCommunities, 1, 2022-

### List Deprecated Datasets
A dataset is deprecated when it is replaced with a newer version or when it is determined that there is no longer a need for the data. Generally, the endpoint remains for about 3 months (as defined in the depDate field) before it is eventually removed. Links to an alternate or new dataset are included along with a reason for the deprecation.

<div class="alert alert-block alert-info">
    <b>Tip:</b> If you want to be alerted to dataset deprecations (as well as OpenFEMA system outages, new datasets, and new API features), <a href="https://public.govdelivery.com/accounts/USDHSFEMA/signup/31270">Subscribe to GovDelivery for OpenFEMA</a> to receive automatic email notifications.
</div>


In [18]:
# declare a URL handling module
import urllib.request
import json

# define URL for the Data Sets endpoint
baseUrl = "https://www.fema.gov/api/open/v1/DataSets"

# include only deprecated datasets
filter = "&$filter=depDate%20ne%20null"    

# some of the more useful attributes
select = "$select=name,version,depDate,depApiMessage,depNewURL"

# we don't really need to see the query metadata, but let's at least sort by category and name
other = "&$metadata=off&$orderby=depDate,name"   

# define the request and read the data 
request = urllib.request.urlopen(baseUrl + "?" + select + filter + other)
result = request.read()

# transform result to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# display results
print(json.dumps(jsonData, indent=2))

{
  "DataSets": [
    {
      "name": "HazardMitigationGrantProgramPropertyAcquisitions",
      "version": 1,
      "depDate": "2019-01-01T00:00:00.000Z",
      "depApiMessage": "This OpenFEMA dataset will no longer be updated and has been archived.",
      "depNewURL": "https://www.fema.gov/about/openfema/data-sets"
    },
    {
      "name": "HazardMitigationGrants",
      "version": 1,
      "depDate": "2019-01-01T00:00:00.000Z",
      "depApiMessage": "This OpenFEMA dataset will no longer be updated and will be archived.",
      "depNewURL": "https://www.fema.gov/about/openfema/data-sets"
    },
    {
      "name": "DisasterDeclarationsSummaries",
      "version": 1,
      "depDate": "2020-04-30T00:00:00.000Z",
      "depApiMessage": "This dataset has been deprecated and will no longer be available by the date specified. It is recommended that the new endpoint be used. See the OpenFEMA documentation for additional information.",
      "depNewURL": "https://www.fema.gov/openfema-dat

### Check for Updates to a Dataset
[API_Tutorial_Part_5_GettingDatasetUpdates](https://github.com/FEMA/openfema-samples/blob/master/analysis-examples/API_Tutorial_Part_4_GettingDatasetUpdates.ipynb) of the tutorial covers this topic in detail. The basic idea is to only pull data 1) if the dataset has been refreshed since your last update (using lastDatasetRefresh, and 2) retrieve only those records that changed using lastRefresh. See the next tutorial for more details and thorough code examples. 

<div class="alert alert-block alert-warning">
    <b>Note:</b> Not every dataset contains the lastRefresh field. More information can be found in the referenced tutorial.
</div>

### Identify a Dataset's Primary Keys
Some of the datasets contain source system fields that can uniquely identify records. Many do not - either they are deemed sensitive or the dataset contains data from many source tables where including a primary key is not possible or practical. OpenFEMA does add an "id" field that can be used to uniquely identify records within one downloaded instance of a dataset. However, this "id" is reset when a dataset is fully reloaded (see more detail in the API documentation ([Special Dataset Fields](https://www.fema.gov/about/openfema/api#Special+Dataset+Fields:~:text=Special%20Dataset%20Fields)). It cannot be used to join with other datasets.

The DataSetFields endpoint can be used to determine what fields, if any, contribute to a primary key.

In [21]:
# declare a URL handling module
import urllib.request
import json

# define URL for the Data Sets endpoint 
baseUrl = "https://www.fema.gov/api/open/v1/DataSetFields"

# limit to the dataset and version for which you want data
filter = "$filter=openFemaDataSet%20eq%20%27DisasterDeclarationsSummaries%27%20and%20datasetVersion%20eq%202%20and%20primaryKey%20eq%20true"    

# only need to return the PK field names
select = "&$select=name"

# define the request and read the data 
request = urllib.request.urlopen(baseUrl + "?" + filter + select)
result = request.read()

# transform result to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# display the keys - if any
for rec in jsonData['DataSetFields']:
    print(rec['name'])

placeCode
disasterNumber


### Produce Valid Data Definition Language (DDL)
Imagine a scenario in which you want to add an OpenFEMA dataset to your own database. Use the metadata endpoints to produce valid DDL/SQL to create the database table. The following code is a start. Most databases will support basic ANSI standard DDL. However, most handle comments associated to fields and tables differently. It would not be hard to modify this script to your use. Indexes can also be created.

<div class="alert alert-block alert-warning">
    <b>Note:</b> This will not work with datasets that are hierarchal in nature such as IPAWS or FEMA Regions.
</div>

In [1]:
# declare a URL handling module
import urllib.request
import json

datasetName = "DisasterDeclarationsSummaries"

# define URL for the Data Sets endpoint and a subsequent query
baseUrl = "https://www.fema.gov/api/open/v1/DataSetFields"

# limit to the dataset and version that you want data for
filter = "&$filter=openFemaDataSet%20eq%20%27" + datasetName + "%27%20and%20datasetVersion%20eq%202"    

# no select - return all the attributes
select = ""

# lets match the order in which fields are presented in OpenFEMA
other = "&$metadata=off&$orderby=sortOrder"

# define the request and read the data 
request = urllib.request.urlopen(baseUrl + "?" + filter + select + other)
result = request.read()

# transform result to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# start to craft the SQL statement
sSQL = "CREATE TABLE " + datasetName + "\n\t("

# initialize primary key
sPK = ""

# cycle through the fields and use attributes to craft SQL
for rec in jsonData['DataSetFields']:
    sSQL = sSQL + rec['name'] + " " + rec['type'] + (" NULL" if rec['isNullable'] else " NOT NULL") + ",\n\t"
    
    if (rec['primaryKey']):
        sPK = sPK + rec['name'] + ", "

# primary key (if any)
if (sPK != ""):
    sSQL = sSQL[:-1] + "CONSTRAINT pk_" + datasetName + " PRIMARY KEY (" + sPK[:-2] + ")\n);"
else:
    # terminate SQL
    sSQL = sSQL[:-2] + ");"
    
# display
print(sSQL)

CREATE TABLE DisasterDeclarationsSummaries
	(femaDeclarationString text NOT NULL,
	disasterNumber smallint NOT NULL,
	state text NOT NULL,
	declarationType text NOT NULL,
	declarationDate date NOT NULL,
	fyDeclared smallint NOT NULL,
	incidentType text NOT NULL,
	declarationTitle text NOT NULL,
	ihProgramDeclared boolean NOT NULL,
	iaProgramDeclared boolean NOT NULL,
	paProgramDeclared boolean NOT NULL,
	hmProgramDeclared boolean NOT NULL,
	incidentBeginDate date NOT NULL,
	incidentEndDate date NULL,
	disasterCloseoutDate date NULL,
	fipsStateCode smallint NOT NULL,
	fipsCountyCode text NOT NULL,
	placeCode text NOT NULL,
	designatedArea text NOT NULL,
	declarationRequestNumber smallint NOT NULL,
	lastIAFilingDate date NULL,
	lastRefresh date NOT NULL,
	hash text NOT NULL,
	id uuid NOT NULL,
CONSTRAINT pk_DisasterDeclarationsSummaries PRIMARY KEY (disasterNumber, placeCode)
);


### List Codes for a Dataset, Dataset Field, and Dataset Value

List all the codes for the Mission Assignments dataset:

In [30]:
# declare a URL handling module
import urllib.request
import json

# define URL for the Data Set Codes endpoint 
baseUrl = "https://www.fema.gov/api/open/v1/DataSetCodes"

# codes for version 2 of the mission assignments dataset
filter = "$filter=datasetName%20eq%20%27MissionAssignments%27%20and%20datasetVersion%20eq%202"

other = "&$metadata=off"    

# define the request and read the data 
request = urllib.request.urlopen(baseUrl + "?" + filter + other)
result = request.read()

# transform result to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# display as a list for readability
for rec in jsonData['DataSetCodes']:
    print( rec['datasetName'] + ', ' + str(rec['datasetVersion']) + ', ' + rec['fieldName'] + ', ' + rec['code'] + ', ' + rec['description'])

MissionAssignments, 2, declarationType, FM, Fire Management Assistance Declaration
MissionAssignments, 2, declarationType, DR, Major Disaster Declaration
MissionAssignments, 2, declarationType, EM, Emergency Declaration
MissionAssignments, 2, declarationType, SU, Surge Activity
MissionAssignments, 2, maType, FOS, Federal Operations Support
MissionAssignments, 2, maType, DFA, Direct Federal Assistance
MissionAssignments, 2, maType, TA, Technical Assistance (no longer used)
MissionAssignments, 2, supportFunction, 0, Other
MissionAssignments, 2, supportFunction, 1, Transportation
MissionAssignments, 2, supportFunction, 2, Communications
MissionAssignments, 2, supportFunction, 3, Public Works and Engineering
MissionAssignments, 2, supportFunction, 4, Firefighting
MissionAssignments, 2, supportFunction, 5, Information and Planning
MissionAssignments, 2, supportFunction, 6, Mass Care, Emergency Assistance, Temporary Housing and Human Services
MissionAssignments, 2, supportFunction, 7, Logist

We can modify the criteria above to narrow the scope to return just the codes for the maType field:

In [31]:
# declare a URL handling module
import urllib.request
import json

# define URL for the Data Set Codes endpoint 
baseUrl = "https://www.fema.gov/api/open/v1/DataSetCodes"

# codes for version 2 of the mission assignments dataset
filter = "$filter=datasetName%20eq%20%27MissionAssignments%27%20and%20datasetVersion%20eq%202"

# add criteria for the maType field only
filter = filter + "%20and%20fieldName%20eq%20%27maType%27"
other = "&$metadata=off"    

# define the request and read the data 
request = urllib.request.urlopen(baseUrl + "?" + filter + other)
result = request.read()

# transform result to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# display as a list for readability - dont show dataset and version
for rec in jsonData['DataSetCodes']:
    print( rec['fieldName'] + ', ' + rec['code'] + ', ' + rec['description'])

maType, DFA, Direct Federal Assistance
maType, FOS, Federal Operations Support
maType, TA, Technical Assistance (no longer used)


In [ ]:
And once again to display just one of the codes:

In [36]:
# declare a URL handling module
import urllib.request
import json

# define URL for the Data Set Codes endpoint 
baseUrl = "https://www.fema.gov/api/open/v1/DataSetCodes"

# codes for version 2 of the mission assignments dataset
filter = "$filter=datasetName%20eq%20%27MissionAssignments%27%20and%20datasetVersion%20eq%202"

# add criteria for the maType field and the FOS code only
filter = filter + "%20and%20fieldName%20eq%20%27maType%27%20and%20code%20eq%20%27FOS%27"

other = "&$metadata=off"    

# define the request and read the data 
request = urllib.request.urlopen(baseUrl + "?" + filter + other)
result = request.read()

# transform result to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# display as a list for readability - dont show dataset and version
for rec in jsonData['DataSetCodes']:
    print( rec['fieldName'] + ', ' + rec['code'] + ', ' + rec['description'])

maType, FOS, Federal Operations Support


## Where to go Next
OpenFEMA API Tutorial: Part 5 - Getting Dataset Updates will demonstrate how to check for dataset and record updates for an OpenFEMA dataset.

## Other Resources
- [OpenFEMA Homepage](https://www.fema.gov/open)
- [OpenFEMA API Documentation](https://www.fema.gov/about/openfema/api)
- [OpenFEMA Samples on GitHub](https://github.com/FEMA/openfema-samples)
- [DCAT-US Schema v1.1 (Project Open Data Metadata Schema)](https://resources.data.gov/schemas/dcat-us/v1.1/)
- [Data.gov](https://www.data.gov/)
- [Dublin Core Metadata Initiative](https://www.dublincore.org/specifications/dublin-core/)
- [GovDelivery for OpenFEMA Notifications](https://public.govdelivery.com/accounts/USDHSFEMA/signup/31270)